In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# read content of book
book_raw = pd.read_csv("../data/processed/Prinz_raw.csv")

# give meta data about book

# copy raw content as a list to fist column second row   
book = book_raw.copy()

In [ ]:
## layout of book = [{'raw':[], 'meta': {}, 'word_list': [], 'word_bag': {}, '2-gram': [], '3-gram': []}] in seperate csv saved at the end of the notebook
# Note - per Hand entfert: , " da Probleme mit pd.read_csv
# Note - per Hand bereinigt: Inhaltsverzeichnis, erste letzte Seite, Kopf und Fußzeilen

In [ ]:
## tidy book

# convert everything to string
book = book.applymap(str)           # it should be string anyway

# save concat all rows in book to get a very long sentence without line breaks
sentence = ''.join(book['content'].tolist())

# remove: numbers
sentence = ''.join([i for i in sentence if not i.isdigit()])

# remove . : ! ? " - ;  etc pp
sentence = sentence.replace('.','')
sentence = sentence.replace(':','')
sentence = sentence.replace('!','')
sentence = sentence.replace('?','')
sentence = sentence.replace('"','')
sentence = sentence.replace('-','')
sentence = sentence.replace('–','')
sentence = sentence.replace(';','')
sentence = sentence.replace('(','')
sentence = sentence.replace(')','')
sentence = sentence.replace('=','')
sentence = sentence.replace('+','')
sentence = sentence.replace('$','')
sentence = sentence.replace('§','')
sentence = sentence.replace('%','')
sentence = sentence.replace('&','')
sentence = sentence.replace('/','')
sentence = sentence.replace('*','')
sentence = sentence.replace('_','')
sentence = sentence.replace('<','')
sentence = sentence.replace('>','')
sentence = sentence.replace('#','')
sentence = sentence.replace('\'','')
sentence = sentence.replace('“','')
sentence = sentence.replace('…','')

# lower case everyting
sentence = sentence.lower()

print(sentence)

In [ ]:
## book_list
book_lst = list(sentence.split())
alnumcheck = [book_lst[i].isalnum() for i in range(len(book_lst))]
book_list = pd.DataFrame(book_lst)
book_list = book_list.rename(columns={0: "list"})

In [ ]:
## book_bag
book_bag = {}

for e in book_lst:
    if e in book_bag:
        book_bag[e] += 1
    else:
        book_bag.update({e: 1})

book_bag = pd.DataFrame(book_bag.items(), columns=['word','freq'])

In [ ]:
## book_meta: titel, author, pages, number of all words, number of unique words, names (list), 

unique = book_list['list'].nunique()

In [ ]:
## book_2gram

In [ ]:
## book_3gram

In [ ]:
## book_05gram

In [ ]:
book_list.to_csv("../data/processed/Prinz_list.csv", index = False)
book_bag.to_csv("../data/processed/Prinz_bag.csv", index = False)
book_meta.to_csv("../data/processed/Prinz_meta.csv", index = False)
book_2gram.to_csv("../data/processed/Prinz_2gram.csv", index = False)
book_3gram.to_csv("../data/processed/Prinz_3gram.csv", index = False)
book_05gram.to_csv("../data/processed/Prinz_05gram.csv", index = False)